In [8]:
from adaptation import MethodSignature, InterfaceSpecification, AdaptationHandler, execute_test, create_adapted_module
from stimulus_sheet_reader import get_stimulus_sheet
from solr_parser import parse_solr_response
from solr_query import translate_to_solr_query
import pysolr

In [9]:
icubed = MethodSignature("log", "float", ["int", "int"])
iminus = MethodSignature("sqrd", "str", ["int"])
interfaceSpecification = InterfaceSpecification("Calculator", [], [icubed, iminus])

In [10]:
solr_url = "http://localhost:8983/solr/lasso_quickstart"
solr = pysolr.Solr(solr_url)
solr_query = translate_to_solr_query(interfaceSpecification)
print("QUERY:", solr_query)
results = solr.search(solr_query)
print(f"Found {len(results)} results")
moduleUnderTest = parse_solr_response(results)

QUERY: (name:log~0.1 AND (arguments.datatype:('int') AND arguments.datatype:('int'))) OR (name:log~0.1 AND count_positional_args:(2)) OR (name:sqrd~0.1 AND (arguments.datatype:('int'))) OR (name:sqrd~0.1 AND count_positional_args:(1))&group=true&group.field=module
Found 3 results
FunctionSignature(functionName='sqrt', returnType='Any', parameterTypes=['Any'], parentClass='None', firstDefault=1)
FunctionSignature(functionName='logn', returnType='Any', parameterTypes=['Any', 'Any'], parentClass='None', firstDefault=2)
Only considering the first module, stopping parsing.
{}


In [11]:
adaptationHandler = AdaptationHandler(interfaceSpecification, moduleUnderTest, excludeClasses=False, useFunctionDefaultValues=False)
adaptationHandler.identifyAdaptations(maxParamPermutationTries=2)
adaptationHandler.visualizeAdaptations()


-----------------
IDENTIFY MAPPINGS
-----------------

               sqrt                       logn
log           None  [NmRtrnCnvr, NmRtrnPrmt*]
sqrd  [NmRtrnCnvr]                       None 



In [12]:
adaptationHandler.generateMappings(onlyKeepTopN=10)

Generated 2 potential mappings.
('log', 'logn', 1)('sqrd', 'sqrt', 0) | score=13
('log', 'logn', 0)('sqrd', 'sqrt', 0) | score=9


In [13]:
(adapted_module, number_of_submodules, submodules_metadata)  = create_adapted_module(adaptationHandler, moduleUnderTest.moduleName, use_constructor_default_values=True)


-----------------------------
TRYING ADAPTATION FOR MAPPING ('log', 'logn', 1)('sqrd', 'sqrt', 0) | score=13.
-----------------------------
Trying to adapt return type of <function logn at 0x7fc551d755e0> to float.
Trying to blindly adapt parameter order of <function logn at 0x7fc551d755e0>.
Created adapted wrapper function.
Adapted name of function <function adapt_function.<locals>.decorator.<locals>.wrapper at 0x7fc5535f3670> to log.
Trying to adapt return type of <function sqrt at 0x7fc551d753a0> to str.
Trying to adapt parameter types of <function sqrt at 0x7fc551d753a0> to ['Any'].
Created adapted wrapper function.
Adapted name of function <function adapt_function.<locals>.decorator.<locals>.wrapper at 0x7fc5535f3700> to sqrd.
Successful creation of submodule 0 for this mapping.

-----------------------------
TRYING ADAPTATION FOR MAPPING ('log', 'logn', 0)('sqrd', 'sqrt', 0) | score=9.
-----------------------------
Trying to adapt return type of <function logn at 0x7fc551d755e0>

In [14]:
stimulus_sheet = get_stimulus_sheet("calc4.csv")
execute_test(stimulus_sheet, adapted_module, number_of_submodules, submodules_metadata)


Executing stimulus sheet on numpy.lib.scimath


Results from executing stimulus sheet:
0 {'log': ('logn', 1, NmRtrnPrmt*), 'sqrd': ('sqrt', 0, NmRtrnCnvr)}		 ('sqrd(7.0)', '2.6457513110645907') ('log(3.0, 2.0)', 1.5849625007211563)
1 {'log': ('logn', 0, NmRtrnCnvr), 'sqrd': ('sqrt', 0, NmRtrnCnvr)}		 ('sqrd(7.0)', '2.6457513110645907') ('log(3.0, 2.0)', 0.6309297535714574)
